# Variational Autoencoder

The variational autoencoder (VAE) aims to learn the distribution of a dataset $\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n$ given they are sampled conditioned on a latent $\mathbf{z}$ sampled as a standard Gaussian vector. Below, we show a basic implementation of a VAE that will be the basis for unsupervised generation down the road.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import random
from tqdm import tqdm
import torchvision
import torchvision.datasets as datasets

#for consistency, all seeds are set to 69420
seed = 69420
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Encoder

The encoder aims to model $q(\mathbf{z}|\mathbf{x};\phi)$. This network produces the mean and covariance using $\mathbf{x}$, from where $\mathbf{z}$ can be sampled. The implementation uses convolutional neural network components to map $\mathbf{x}$ to the means and covariances. A trick used here is to sample the covariance from the logarithmic space to ensure positivity.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_size, img_channel):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channel, 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(32, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
        )
        self.mu_fc = nn.Conv2d(256, latent_size, 2, 1, 0, bias=False)
        self.sigma_fc = nn.Conv2d(256, latent_size, 2, 1, 0, bias=False)
        self.kl = 0
        
    def forward(self,x,normal_generator):
        encoded = self.encoder(x)
        mu = self.mu_fc(encoded).mean([-2,-1]) #mean
        sigma = torch.exp(self.sigma_fc(encoded).mean([-2,-1])) #covariance
        z = mu + sigma*normal_generator.sample(mu.shape) #randomly sample from latent
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum() # kl loss term
        return z

# Decoder

On the other hand, the decoder learns $p(\mathbf{x}|\mathbf{z};\theta)$ by reconstructing the give data $x$ using information it was given from $\mathbf{z}$. Implementation-wise, this decoder uses convolution transpose blocks to reverse the convolution of the encoder.

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self, latent_size, img_channel):
        
        super(Decoder, self).__init__()

        self.conv_transpose_block_1 = nn.Sequential(
            nn.ConvTranspose2d(latent_size, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True))

        self.conv_transpose_block_2 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True))

        self.conv_transpose_block_3 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True))

        self.conv_transpose_block_4 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True))

        self.conv_transpose_block_5 = nn.Sequential(
            nn.ConvTranspose2d(32, img_channel, 1, 1, 0, bias=False),
            nn.Sigmoid())

    def forward(self, x):
        x = self.conv_transpose_block_1(x)
        x = self.conv_transpose_block_2(x)
        x = self.conv_transpose_block_3(x)
        x = self.conv_transpose_block_4(x)
        x = self.conv_transpose_block_5(x)
        return x

## VAE training
We optimize the ELBO stochastically using the gradient descent, which boils down the minimizing the KL divergence and 